In [1]:
import mediapipe as mp
import cv2
import imutils
import numpy as np
from sklearn.metrics import pairwise

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
bg = None

def run_avg(image, accumWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return
    
    cv2.accumulateWeighted(image, bg, accumWeight)

def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:

        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

def count(thresholded, segmented):

    chull = cv2.convexHull(segmented)

    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])

    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)


    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]

    radius = int(0.8 * maximum_distance)

    circumference = (2 * np.pi * radius)

    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")

    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)

    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    (cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    count = 0

    for c in cnts:

        (x, y, w, h) = cv2.boundingRect(c)

        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1

    return count

if __name__ == "__main__":
    
    accumWeight = 0.5

    camera = cv2.VideoCapture(0)

    top, right, bottom, left = 10, 350, 225, 590

    num_frames = 0

    calibrated = False
    
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while(True):

        (grabbed, frame) = camera.read()
        
        feed = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Make Detection
        results = holistic.process(feed)
        #print(results.face_landmarks)
        
        #Recolor feed
        feed = cv2.cvtColor(feed, cv2.COLOR_RGB2BGR)
        
        mp_drawing.DrawingSpec(color = (0, 0, 255), thickness= 2, circle_radius= 2)
        
        #Draw face landmarks
        mp_drawing.draw_landmarks(feed, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                                                              mp_drawing.DrawingSpec(color= (80, 110, 109), thickness= 1, circle_radius= 1),
                                                              mp_drawing.DrawingSpec(color= (80, 256, 121), thickness= 1, circle_radius= 1)
                                                              )
        
        #Right hand
        mp_drawing.draw_landmarks(feed, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                                              mp_drawing.DrawingSpec(color= (80, 22, 10), thickness= 2, circle_radius= 4),
                                                              mp_drawing.DrawingSpec(color= (80, 44, 121), thickness= 2, circle_radius= 2)
                                                              )
        
        #Left hand
        mp_drawing.draw_landmarks(feed, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                              mp_drawing.DrawingSpec(color= (121, 22, 76), thickness= 2, circle_radius= 4),
                                                              mp_drawing.DrawingSpec(color= (121, 44, 250), thickness= 2, circle_radius= 2)
                                                              )
        
        #Pose Detections
        mp_drawing.draw_landmarks(feed, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                                              mp_drawing.DrawingSpec(color= (245, 117, 66), thickness= 2, circle_radius= 4),
                                                              mp_drawing.DrawingSpec(color= (245, 66, 230), thickness= 2, circle_radius= 2)
                                                              )
        
        feed = cv2.flip(feed, 1)

        clone = feed.copy()

        (height, width) = feed.shape[:2]

        roi = feed[top:bottom, right:left]

        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:

            hand = segment(gray)

            if hand is not None:

                (thresholded, segmented) = hand

                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                fingers = count(thresholded, segmented)

                cv2.putText(clone, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
 
                cv2.imshow("Thesholded", thresholded)

        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        num_frames += 1

        cv2.imshow("Video Feed", clone)

        keypress = cv2.waitKey(1) & 0xFF

        if keypress == ord("q"):
            break

camera.release()
cv2.destroyAllWindows()

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
